In [2]:
import pandas as pd
from itertools import chain
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from collections import Counter

In [4]:
data_dir = '../synonyms_from_category/data/'
dt = '20190130'

In [5]:
df_synonyms = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/candidate_synonyms_final_190130.tsv', sep='\t', names=['word', 'synonyms'])


In [6]:
df_users = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/cate_WE_user_dict_190130.txt', names=['word'])

In [7]:
df = df_synonyms.append(df_users, ignore_index=True)

/home/admin-/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [8]:
df = df.sort_values(by=['word']).reset_index(drop=True)
df.synonyms = df.synonyms.astype('str')
df.synonyms = df.synonyms.fillna(value=0)
type(df.synonyms[1])

str

In [9]:
df['output'] = None


def word_and_synonyms(row):
    temp_str = row['word']
    if row['synonyms'] != 'nan':
        temp_str = temp_str + " " + row['synonyms']
        temp_list = temp_str.split(" ")
    else:
        temp_list = [temp_str]
    row['output'] = temp_list
    return row


df = df.apply(lambda row: word_and_synonyms(row), axis=1)

In [10]:
df.output.sample(20)

215                              [도톰, 도툼, 두툼]
750                       [스티커, sticker, 스티카]
538                                   [분유케이스]
717                                   [스노우체인]
1461                                   [현대소설]
299                              [리필, refill]
1141                                    [착용감]
772                           [시트지, 쉬트지, 씨트지]
1366                                   [패밀리룩]
1128                                    [진미채]
99      [귀걸이, earing, 귀고리, 이어링, earring, 귀거리]
1046                                    [자전거]
694                                    [수건걸이]
276            [로션, lotion, 로숀, 로시옹, 로씨옹, 료션]
1226                                    [커튼봉]
627                                     [선반장]
346                               [면러그 코튼 러그]
138                                    [남태평양]
1172                                    [초경량]
621                           [서랍장, 서랖장, 선랍장]
Name: output, dtype: object

In [11]:
list_output = df.output.tolist()
list_output = [_.lower() for _ in chain(*list_output)]

In [12]:
print(len(pd.Series(list_output)), len(pd.Series(list_output).unique()))

2841 2722


In [13]:
pd.Series(list_output).to_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/freq_user_dict_'+dt+'.txt', index=False)